In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [195]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


###Tratamento dos Dados

In [168]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [169]:
# transformando valores categoricos em numericos
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [0]:
def transformValues(valor):
  if valor == 'female':
    return 0
  else:
    return 1

train['sex_bin'] = train['Sex'].map(transformValues)

In [171]:
# preenchendo os missing values da coluna 'Age' com a média das idades registrdas
media = round(np.mean(train['Age']))
media

30

In [0]:
train['Age'] = train['Age'].fillna(media)

In [0]:
# excluindo a coluna 'Cabin'
train = train.drop(columns='Cabin')

In [0]:
# excluindo linhas com valores faltantes
train.dropna(inplace=True)

Tratando a variavel 'Name'

In [175]:
# podemos utilizar expressões regulares para tratar a variavel 'Name'
exp_tit = " ([^. ]+\.)"
train['Prefix'] = train['Name'].str.extract(exp_tit)
train[['Prefix', 'Name']].head()

,Prefix,Name
0,Mr.,"Braund, Mr. Owen Harris"
1,Mrs.,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,Miss.,"Heikkinen, Miss. Laina"
3,Mrs.,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,Mr.,"Allen, Mr. William Henry"


In [176]:
# contagem dos titulos
train['Prefix'].value_counts()

Mr.          517
Miss.        181
Mrs.         124
Master.       40
Dr.            7
Rev.           6
Mlle.          2
Col.           2
Major.         2
Jonkheer.      1
Mme.           1
Lady.          1
Sir.           1
Countess.      1
Don.           1
Capt.          1
Ms.            1
Name: Prefix, dtype: int64

In [177]:
# podemos reorganizar os titulos de forma resumida
titulos = {
    "Capt.": "Officer",
    "Col.": "Officer",
    "Major.": "Officer",
    "Dr.": "Officer",
    "Rev.": "Officer",
    "Jonkheer.": "Royalty",
    "Don.": "Royalty",
    "Sir.": "Royalty",
    "Countess.": "Royalty",
    "Lady.": "Royalty",
    "Mr.": "Mr",
    "Mrs.": "Mrs",
    "Ms.": "Mrs",
    "Mme.": "Mrs",
    "Miss.": "Miss", 
    "Mlle.": "Miss",
    "Master.": "Master",
    "Dona.": "Mrs",
}

def substituir_titulos(linha):
  return titulos[linha]

train['Title'] = train['Prefix'].apply(substituir_titulos)
train['Title'].value_counts()

Mr         517
Miss       183
Mrs        126
Master      40
Officer     18
Royalty      5
Name: Title, dtype: int64

Tratando a variavel 'Ticket'

In [178]:
# podemos fazer o agrupamento dos tickets de acordo com sua numeração
df_ticket = train.groupby(['Ticket'])[['Name']].count()
df_ticket.sort_values('Name', ascending=False).head(10)

,Name
Ticket,
CA. 2343,7
347082,7
1601,7
347088,6
CA 2144,6
3101295,6
382652,5
S.O.C. 14879,5
19950,4


In [179]:
# renomeando a coluna 'Name'
df_ticket = df_ticket.rename(columns={'Name' : 'TicketCount'})
df_ticket.head()

,TicketCount
Ticket,
110152,3
110413,3
110465,2
110564,1
110813,1


In [0]:
# agora podemos inserir esses dados no DataFrame train
train = pd.merge(train, df_ticket, left_on = 'Ticket', right_index= True).sort_index()

Trabalhando com outras colunas

In [0]:
# podemos somas os valores das colunas 'SibSP' e 'Parch' para determinar o tamanho da familia
train['familysize'] = train['SibSp'] + train['Parch'] + 1

In [0]:
# podemos multiplicar os valores das colunas 'familysize' e 'fare' para encontrar o valor total da tarifa/familia
train['familyfare'] = train['familysize'] * train['Fare']

In [0]:
# vamos criar uma coluna para identificar passageiros com menos de 16 anos
train["child"] = train["Age"] < 16

In [0]:
# identificando passageiros que viajam sozinhos
train["noRelatives"] = train["familysize"] == 0

In [185]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,sex_bin,Prefix,Title,TicketCount,familysize,familyfare,child,noRelatives
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,Mr.,Mr,1,2,14.5000,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,0,Mrs.,Mrs,1,2,142.5666,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0,Miss.,Miss,1,1,7.9250,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,0,Mrs.,Mrs,2,2,106.2000,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1,Mr.,Mr,1,1,8.0500,False,False


In [186]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
sex_bin        0
Prefix         0
Title          0
TicketCount    0
familysize     0
familyfare     0
child          0
noRelatives    0
dtype: int64

In [187]:
colunas_binarias = ["Sex", 'child', 'noRelatives']
colunas_categoricas = ["Pclass", "Embarked", "Title"]
colunas_numericas = ["Age", "SibSp", "Parch", "Fare", "TicketCount", "familysize", "familyfare"]

def processar_dataframe(df, colunas_binarias, colunas_categoricas, colunas_numericas):
    df_saida = pd.get_dummies(df[colunas_categoricas].astype(str))
    for col in colunas_binarias:
        if df[col].dtype == bool:
            df_saida[col] = df[col]
        else:
            sr_bin = df[col]
            prefixo = df[col][0]
            df_saida[col+"_"+prefixo] = sr_bin == prefixo
    df_saida = pd.concat([df_saida, df[colunas_numericas]], axis=1)
    return df_saida


X = processar_dataframe(train, colunas_binarias, colunas_categoricas, colunas_numericas)
y = train["Survived"]
X.head()

,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Sex_male,child,noRelatives,Age,SibSp,Parch,Fare,TicketCount,familysize,familyfare
0,0,0,1,0,0,1,0,0,1,0,0,0,True,False,False,22.0,1,0,7.2500,1,2,14.5000
1,1,0,0,1,0,0,0,0,0,1,0,0,False,False,False,38.0,1,0,71.2833,1,2,142.5666
2,0,0,1,0,0,1,0,1,0,0,0,0,False,False,False,26.0,0,0,7.9250,1,1,7.9250
3,1,0,0,0,0,1,0,0,0,1,0,0,False,False,False,35.0,1,0,53.1000,2,2,106.2000
4,0,0,1,0,0,1,0,0,1,0,0,0,True,False,False,35.0,0,0,8.0500,1,1,8.0500


In [203]:
X.shape

(889, 22)

### Treinando o Algorítmo

In [0]:
# instanciando o objeto Random Forest
model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

In [189]:
# treinando o modelo com os dados reduzidos
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [190]:
# gerando as predições
train['predicoes'] = model.predict(X)

# medindo a acuracia
acc = metrics.accuracy_score(
    train['Survived'],
    train['predicoes']
)

acc

0.9820022497187851

### Testando o Algorítmo

In [0]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Titanic/test.csv')

In [212]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
# transformando valores categoricos em numericos
test['sex_bin'] = test['Sex'].map(transformValues)
test['Age'] = test['Age'].fillna(media)

test = test.drop(columns='Cabin')

test.dropna(inplace=True)

test['Prefix'] = test['Name'].str.extract(exp_tit)
test['Title'] = test['Prefix'].apply(substituir_titulos)

df_ticket_teste = test.groupby(["Ticket"])[["Name"]].count()
df_ticket_teste = df_ticket_teste.rename(columns={"Name": "TicketCount"})
test = pd.merge(test, df_ticket_teste, left_on="Ticket", right_index=True).sort_index()

test["child"] = test["Age"] < 16

test['familysize'] = test['SibSp'] + test['Parch'] + 1
test['familyfare'] = test['familysize'] * test['Fare']
test["noRelatives"] = test["familysize"] == 0

In [214]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,sex_bin,Prefix,Title,TicketCount,child,familysize,familyfare,noRelatives
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,1,Mr.,Mr,1,False,1,7.8292,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,0,Mrs.,Mrs,1,False,2,14.0000,False
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,1,Mr.,Mr,1,False,1,9.6875,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,1,Mr.,Mr,1,False,1,8.6625,False
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,0,Mrs.,Mrs,1,False,3,36.8625,False


In [216]:
X_teste1 = test[["Age", "SibSp", "Parch", "Fare"]]

X_teste2 = processar_dataframe(test, colunas_binarias, colunas_categoricas, colunas_numericas)
X_teste2["Title_Royalty"] = 0
X_teste2 = X_teste2[X.columns]
X_teste2.head()

,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Sex_male,child,noRelatives,Age,SibSp,Parch,Fare,TicketCount,familysize,familyfare
0,0,0,1,0,1,0,0,0,1,0,0,0,True,False,False,34.5,0,0,7.8292,1,1,7.8292
1,0,0,1,0,0,1,0,0,0,1,0,0,False,False,False,47.0,1,0,7.0000,1,2,14.0000
2,0,1,0,0,1,0,0,0,1,0,0,0,True,False,False,62.0,0,0,9.6875,1,1,9.6875
3,0,0,1,0,0,1,0,0,1,0,0,0,True,False,False,27.0,0,0,8.6625,1,1,8.6625
4,0,0,1,0,0,1,0,0,0,1,0,0,False,False,False,22.0,1,1,12.2875,1,3,36.8625


In [217]:
# fazendo as previsões...
p = model.predict(X_teste2)
p

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [0]:
# alinhando os valores previstos ao dataset test
sub = pd.Series(p, index=test['PassengerId'], name='Survived')

In [0]:
# salvando como arquivo csv
sub.to_csv('primeiro_envio.csv', header=True)